# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
#print pysd.__version__
print pysd.__file__
import pandas as pd
import os
import xarray as xr

Populating the interactive namespace from numpy and matplotlib
pysd/__init__.pyc


/Users/houghton/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
model = pysd.read_vensim('./tests/test-models/tests/abs/test_abs.mdl')
model

ValueError: shape mismatch: value array of shape (3,2) could not be broadcast to indexing result of shape (1,2,3)

In [3]:
xr.DataArray(data=[[0.01,0.02],[0.03,0.04],[0.05,0.06]], coords={'Dim2': ['D', 'E'], 
                                                                 'Dim1': ['A', 'B', 'C']}, dims=['Dim1', 'Dim2'] )

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ 0.01,  0.02],
       [ 0.03,  0.04],
       [ 0.05,  0.06]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [5]:
xr.DataArray(data=10, coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}, dims=['Dim1', 'Dim2'] )

KeyError: 'Dim2'

In [8]:
a = xr.DataArray(data=np.ones([3])*10, coords={'Dim1': ['A', 'B', 'C']})

In [9]:
xr.DataArray(data=np.ones([2, 3])*10, coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']})

<xarray.DataArray (Dim2: 2, Dim1: 3)>
array([[ 10.,  10.,  10.],
       [ 10.,  10.,  10.]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [2]:
pysd.read_vensim('./tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl')

ValueError: shape mismatch: value array of shape (3,2) could not be broadcast to indexing result of shape (1,2,3)

In [3]:
%debug

> /Users/houghton/Google_Drive/Academic Projects/PYSD/pysd/pysd/pysd.py(254)initialize_state()
    252                     initialization_order.append(key)
    253                 except KeyError:  # may also need to catch TypeError?
--> 254                     retry_flag = True
    255             if not making_progress:
    256                 raise KeyError('Unresolvable Reference: Probable circular initialization'+

ipdb> model.components
*** NameError: name 'model' is not defined
ipdb> ls
*** NameError: name 'ls' is not defined
ipdb> w
  <ipython-input-2-7798442bc851>(1)<module>()
----> 1 pysd.read_vensim('./tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl')

  /Users/houghton/Google_Drive/Academic Projects/PYSD/pysd/pysd/pysd.py(62)read_vensim()
     60     from vensim2py import translate_vensim
     61     py_model_file = translate_vensim(mdl_file)
---> 62     model = load(py_model_file)
     63     return model
     64 

  /Users/houghton/Goo

In [47]:
np.ones(3)*NaN

array([ nan,  nan,  nan])

In [2]:
import xarray as xr

In [4]:
coords = {'Second Dimension Subscript': ['Column 1', 'Column 2'],
          'Third Dimension Subscript': ['Depth 1', 'Depth 2'],
          'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3']}
coords

{'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3'],
 'Second Dimension Subscript': ['Column 1', 'Column 2'],
 'Third Dimension Subscript': ['Depth 1', 'Depth 2']}

In [95]:
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
a = xr.DataArray(data=np.zeros(map(len, coords.values()))*NaN, coords=coords)

#a.loc[{'Dim2':['D']}] = [1, 2, 3]
#a.loc[coords] = [1,2],[3,4],[5,6]

values = [1,2],[3,4],[5,6]

a.values = np.array(values)

ValueError: replacement data must match the Variable's shape

In [98]:
coords={'Dim1': ['A', 'B', 'C'], 'Dim2': ['D', 'E']}
a = xr.DataArray(data=[[1,2],[3,4],[5,6]], coords=coords, dims=['Dim1', 'Dim2'])


In [43]:
a = xr.DataArray(data=np.zeros(map(len, coords.values()))*NaN, coords=coords)

In [72]:
m = np.empty([3])*NaN
m

array([ nan,  nan,  nan])

In [70]:
np.empty(4)

array([ 1.,  2.,  3.,  4.])

In [49]:
a.sel(One Dimensional Subscript='Column 1')

SyntaxError: invalid syntax (<ipython-input-49-f3f8947e1e9d>, line 1)

In [36]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 1)>
array([[[ 1.],
        [ 2.]],

       [[ 3.],
        [ 4.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1'

In [ ]:
a.loc[{'One Dimensional Subscript': ['Entry 1']}]

In [18]:
a.loc[:,:,'Entry 1']

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2)>
array([[ 1.,  2.],
       [ 0.,  0.]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [37]:
a.loc[:,'Entry 1',:]

KeyError: 'Entry 1'

In [16]:
a.loc['Column 1',:,'Entry 1']

<xarray.DataArray (Third Dimension Subscript: 2)>
array([ 0.,  0.])
Coordinates:
    Second Dimension Subscript  |S8 'Column 1'
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
    One Dimensional Subscript   |S7 'Entry 1'

In [17]:
a.loc['Column 1',:,'Entry 1'] = 1, 2

In [21]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [22]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [25]:
a.loc[:,:,'Entry 1'] = [1,2],[3,4]

In [26]:
a

<xarray.DataArray (Second Dimension Subscript: 2, Third Dimension Subscript: 2, One Dimensional Subscript: 3)>
array([[[ 1.,  0.,  0.],
        [ 2.,  0.,  0.]],

       [[ 3.,  0.,  0.],
        [ 4.,  0.,  0.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...

In [50]:
m = {'Dim1': ['A', 'B'],
                                      'Dim2': ['C', 'D', 'E'],
                                      'Dim3': ['F', 'G', 'H', 'I']}

In [53]:
m.iterkeys()

<dictionary-keyiterator at 0x10d7459f0>

In [37]:
frame = {'elem1': 'hibob'}

In [48]:
frame = {'elem1': xr.DataArray([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                                       {'Dim1': ['A', 'B', 'C'],
                                        'Dim2': ['D', 'E', 'F']}),
         'elem2': xr.DataArray([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                                       {'Dim1': ['A', 'B', 'C'],
                                        'Dim2': ['D', 'E', 'F']})}

return_addresses = {'Elem1': ('elem1', {'Dim1': ['B'], 'Dim2': ['F']})}

In [109]:
pyname = 'elem1',
for real_name, (pyname, address) in return_addresses.iteritems():
    a = frame[pyname].loc[address]

In [120]:
max(a.shape)

1

In [125]:
a.values.shape

(1, 1)

In [110]:
a == 8

<xarray.DataArray (Dim2: 1, Dim1: 1)>
array([[ True]], dtype=bool)
Coordinates:
  * Dim2     (Dim2) |S1 'F'
  * Dim1     (Dim1) |S1 'B'

In [111]:
b = a.values.squeeze()

In [117]:
val = b.squeeze()
val.flatten()

array([8])

In [100]:
b = np.squeeze(a.values)

In [108]:
float(a)

TypeError: only length-1 arrays can be converted to Python scalars

In [42]:
xr.DataArray([[1, 2, 3], [4, 5, 6], [7, 8, 9]],
                                       {'Dim1': ['A', 'B', 'C'],
                                        'Dim2': ['D', 'E', 'F']})

<xarray.DataArray (Dim2: 3, Dim1: 3)>
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E' 'F'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'